In [1]:
import os
os.getcwd()

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
dir = os.path.join('drive','My Drive','Colab Notebooks','STAT628_module3')
dir

'drive/My Drive/Colab Notebooks/STAT628_module3'

### Prepare data

In [0]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [0]:
import torch
from torchtext import data
from torchtext import datasets
from torchtext.data import TabularDataset

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', preprocessing = generate_bigrams)
LABEL = data.LabelField(dtype = torch.float)

In [0]:
txt_datafields = [("text", TEXT), # we won't be needing the id, so we pass in None as the field
                  ("stars", LABEL)]
txt = TabularDataset(
           path=os.path.join(dir,"dataset","RNN_total.csv"), # the file path
           format='csv',
           skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
           fields=txt_datafields)

In [0]:
import random

train_data, other = txt.split(random_state = random.seed(SEED))
valid_data,test_data = other.split(split_ratio = 0.5,random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 86378
Number of validation examples: 18510
Number of testing examples: 18509


In [9]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:29, 2.21MB/s]                          
100%|█████████▉| 399029/400000 [00:21<00:00, 17684.56it/s]

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),  ####  add
    sort_within_batch = False,
    device = device)

In [11]:
for i in test_iterator:
  print(i.text)
  break

100%|█████████▉| 399029/400000 [00:40<00:00, 17684.56it/s]

tensor([[    0,     0, 10486,  ...,    54,    54,  1184],
        [    1,     1,     1,  ...,  1193,    24,    17],
        [    1,     1,     1,  ...,  1048,     4,   145],
        ...,
        [    1,     1,     1,  ...,    81,  1537,     0],
        [    1,     1,     1,  ...,  3049,  6901,  2938],
        [    1,     1,     1,  ..., 18243,     0,   695]], device='cuda:0')


### build the model

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = len(LABEL.vocab)
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [14]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,500,705 trainable parameters


In [15]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.1868, -1.4264, -0.7237,  ...,  0.4038,  2.0948,  0.8120],
        [-0.2089,  0.4165, -0.2762,  ..., -0.1204,  0.3776,  0.7780],
        [-0.8258,  1.6444,  0.9838,  ..., -0.4411,  0.1295, -0.6769]])

In [16]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.1868, -1.4264, -0.7237,  ...,  0.4038,  2.0948,  0.8120],
        [-0.2089,  0.4165, -0.2762,  ..., -0.1204,  0.3776,  0.7780],
        [-0.8258,  1.6444,  0.9838,  ..., -0.4411,  0.1295, -0.6769]])


### train the model

In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    correct = max_preds.squeeze(1).eq(y)
    return correct.sum() / torch.FloatTensor([y.shape[0]])

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.stars.long())
        
        acc = categorical_accuracy(predictions, batch.stars.long())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.stars.long())
            
            acc = categorical_accuracy(predictions, batch.stars.long())

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [23]:
N_EPOCHS = 50

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 33s
	Train Loss: 1.259 | Train Acc: 47.89%
	 Val. Loss: 1.032 |  Val. Acc: 58.84%
Epoch: 02 | Epoch Time: 0m 33s
	Train Loss: 0.988 | Train Acc: 58.06%
	 Val. Loss: 1.277 |  Val. Acc: 62.70%
Epoch: 03 | Epoch Time: 0m 33s
	Train Loss: 0.879 | Train Acc: 63.21%
	 Val. Loss: 1.431 |  Val. Acc: 64.17%
Epoch: 04 | Epoch Time: 0m 33s
	Train Loss: 0.823 | Train Acc: 65.55%
	 Val. Loss: 1.518 |  Val. Acc: 65.09%
Epoch: 05 | Epoch Time: 0m 33s
	Train Loss: 0.786 | Train Acc: 67.22%
	 Val. Loss: 1.589 |  Val. Acc: 65.68%
Epoch: 06 | Epoch Time: 0m 32s
	Train Loss: 0.759 | Train Acc: 68.37%
	 Val. Loss: 1.632 |  Val. Acc: 66.07%
Epoch: 07 | Epoch Time: 0m 32s
	Train Loss: 0.735 | Train Acc: 69.37%
	 Val. Loss: 1.685 |  Val. Acc: 66.06%
Epoch: 08 | Epoch Time: 0m 32s
	Train Loss: 0.714 | Train Acc: 70.40%
	 Val. Loss: 1.738 |  Val. Acc: 66.57%
Epoch: 09 | Epoch Time: 0m 32s
	Train Loss: 0.694 | Train Acc: 71.40%
	 Val. Loss: 1.785 |  Val. Acc: 66.91%
Epoch: 10 | Epoch T

In [24]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.012 | Test Acc: 59.83%


In [0]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    pre_class = prediction.argmax()
    return LABEL.vocab.itos[pre_class]

In [26]:
pre_class1 = predict_sentiment(model, "I like the flavor")
pre_class1

'4.0'

In [27]:
pre_class2 = predict_sentiment(model, "a little expensive")
pre_class2

'3.0'

In [28]:
pre_class2 = predict_sentiment(model, "The service sucks")
pre_class2

'1.0'

## Prediction

In [0]:
import numpy as np
import pandas as pd

In [30]:
food = pd.read_csv(os.path.join(dir,'dataset','food_sentence.csv'))
food.head(2)

,business_id,text
0,irft4YkdNsww4DNf_Aftew,pretty good froyo place with all the toppings ...
1,irft4YkdNsww4DNf_Aftew,very tasty even with the added protein it does...


In [0]:
food_pre = list(map(lambda x: predict_sentiment(model, x),food.text))
#food_pre

In [34]:
food_stars = pd.concat([food,pd.DataFrame({'stars':food_pre})],axis = 1)
food_stars.head()

,business_id,text,stars
0,irft4YkdNsww4DNf_Aftew,pretty good froyo place with all the toppings ...,4.0
1,irft4YkdNsww4DNf_Aftew,very tasty even with the added protein it does...,4.0
2,GWO87Y-IqL54_Ijx6hTYAQ,they have tons of cool options and taste bud b...,5.0
3,GWO87Y-IqL54_Ijx6hTYAQ,one of my favorites is the coconut chip.,5.0
4,GWO87Y-IqL54_Ijx6hTYAQ,not that i'm saying bruster's is bad or anythi...,3.0


In [0]:
food_stars.to_csv(os.path.join(dir,'result','food_stars.csv'),index=False)

In [38]:
price = pd.read_csv(os.path.join(dir,'dataset','price_sentence.csv'))
price_pre = list(map(lambda x: predict_sentiment(model, x),price.text))
price_stars = pd.concat([price,pd.DataFrame({'stars':price_pre})],axis = 1)
price_stars.head()

,business_id,text,stars
0,GWO87Y-IqL54_Ijx6hTYAQ,"the serving size is decent, the texture is spo...",4.0
1,GWO87Y-IqL54_Ijx6hTYAQ,the ice cream is reasonably priced--i am willi...,4.0
2,GWO87Y-IqL54_Ijx6hTYAQ,reasonably priced.,3.0
3,GWO87Y-IqL54_Ijx6hTYAQ,prices are reasonable for the quality and port...,4.0
4,GWO87Y-IqL54_Ijx6hTYAQ,they are also always willing to give a taste i...,4.0


In [0]:
price_stars.to_csv(os.path.join(dir,'result','price_stars.csv'),index=False)

In [40]:
service = pd.read_csv(os.path.join(dir,'dataset','service_sentence.csv'))
service_pre = list(map(lambda x: predict_sentiment(model, x),service.text))
service_stars = pd.concat([service,pd.DataFrame({'stars':service_pre})],axis = 1)
service_stars.head()

,business_id,text,stars
0,irft4YkdNsww4DNf_Aftew,owners are super nice.,5.0
1,irft4YkdNsww4DNf_Aftew,shop is clean and staff is pleasant.,5.0
2,irft4YkdNsww4DNf_Aftew,i've had a lot of bubble teas in charlotte and...,5.0
3,irft4YkdNsww4DNf_Aftew,the owner uses fresh fruit and it makes a huge...,5.0
4,irft4YkdNsww4DNf_Aftew,clean establishment with friendly service,5.0


In [0]:
service_stars.to_csv(os.path.join(dir,'result','service_stars.csv'),index=False)

### groupby mean stars

In [54]:
toy = food_stars.iloc[[10,15,20,25,26],[0,2]]
toy

,business_id,stars
10,GWO87Y-IqL54_Ijx6hTYAQ,4.0
15,GWO87Y-IqL54_Ijx6hTYAQ,5.0
20,GWO87Y-IqL54_Ijx6hTYAQ,5.0
25,kANF0dbeoW34s2vwh6Umfw,1.0
26,kANF0dbeoW34s2vwh6Umfw,4.0


In [75]:
toy.stars = pd.to_numeric(toy.stars)
toy.groupby(['business_id']).mean()

,stars
business_id,
GWO87Y-IqL54_Ijx6hTYAQ,4.666667
kANF0dbeoW34s2vwh6Umfw,2.500000


In [86]:
food_res = food_stars[['business_id','stars']].astype({'stars':np.float64})
food_advice = food_res.groupby(['business_id']).mean()
food_advice.head()

,stars
business_id,
-2Arz8twKJmxHMS3Sv-TeA,4.000000
-36lZCEYtRfW0ZR4Nsem5Q,3.885714
-3H_6UZGWrfl8XQsm1fBmw,3.800000
-7HB0UjguXW8csOH9svG4Q,2.695652
-95mbLJsa0CxXhpaNL4LvA,4.005952


In [88]:
food_advice.shape

(2582, 1)

In [90]:
np.unique(food_res.business_id).shape

(2582,)

In [0]:
food_advice.to_csv(os.path.join(dir,'result','food_advice.csv'),index=False)

In [93]:
price_res = price_stars[['business_id','stars']].astype({'stars':np.float64})
price_advice = price_res.groupby(['business_id']).mean()
price_advice.head()

,stars
business_id,
-2Arz8twKJmxHMS3Sv-TeA,4.000000
-36lZCEYtRfW0ZR4Nsem5Q,3.571429
-7HB0UjguXW8csOH9svG4Q,2.500000
-95mbLJsa0CxXhpaNL4LvA,3.482456
-AgGcbf7oX_MkGGQhaMH0g,2.500000


In [0]:
price_advice.to_csv(os.path.join(dir,'result','price_advice.csv'),index=False)

In [95]:
service_res = service_stars[['business_id','stars']].astype({'stars':np.float64})
service_advice = service_res.groupby(['business_id']).mean()
service_advice.head()

,stars
business_id,
-2Arz8twKJmxHMS3Sv-TeA,3.581395
-36lZCEYtRfW0ZR4Nsem5Q,3.250000
-3H_6UZGWrfl8XQsm1fBmw,3.666667
-7HB0UjguXW8csOH9svG4Q,1.857143
-95mbLJsa0CxXhpaNL4LvA,3.273469


In [0]:
service_advice.to_csv(os.path.join(dir,'result','service_advice.csv'),index=False)

In [98]:
service_advice.shape

(2746, 1)